In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
data = pd.read_csv('../input/pakwheels/updated_pakwheels.csv')
data.head()

In [3]:
data['Features']

In [4]:
data.describe()

In [5]:
data.isnull().sum()

In [6]:
col = list(data.columns)
for i in range(len(col)):
    print(col[i], (data[col[i]].nunique()))

In [7]:
col_to_drop = ['Ad No', 'Last Updated', 'URL']
data.drop(columns = col_to_drop, inplace = True)

In [8]:
data.head()

In [9]:
data.Name = data.Name.apply(lambda x: x.split()[0])
data.head()

In [10]:
data.Location = data.Location.apply(lambda x: x.split()[-1])
data.head()

In [11]:
plt.figure(figsize = (10,10))
sb.histplot(data['Mileage'])
plt.show()
# hence needs to be normalized using MinMaxScaler

In [12]:
scaler= MinMaxScaler()
data['Mileage']
# scaler.fit_transform(data['Mileage'])

In [13]:
# return 'Registered' if i != 'Unregistered' return 'Un-Registered' for i in data['Registered']
# data['Registered'] = data['Registered'].apply(lambda x: 'Registered' if x!= 'Un-Registered' else 'Un-Registered')
# data.head()

def enc_reg(val):
    if val != 'Un-Registered':
        return 'Registered'
    else:
        return 'Un-Registered'

data['Registered City'] = data['Registered City'].apply(enc_reg)
data.head()

In [14]:
data.rename(columns = {"Registered City": "Registered"}, inplace = True)
data.head()

In [15]:
Y = data.Mileage
Y

In [16]:
data.drop(columns = ['Features', 'Mileage'], inplace = True)

In [17]:
col = list(data.columns)
for i in range(len(col)):
    print(col[i], (data[col[i]].nunique()))

In [18]:
data.head()

In [19]:
data.dtypes
type(data)

In [20]:
num = data.select_dtypes(include = ['int64'])
cat = data.select_dtypes(include = ['object'])
cat

In [21]:
cat = pd.get_dummies(cat, drop_first = True)
cat

In [22]:
num

In [23]:
type(cat)

In [24]:
dataf = pd.concat([cat, num], axis = 1)

In [25]:
dataf.info()

In [26]:
X_train, X_test, y_train, y_test = train_test_split(dataf, Y, test_size = 0.25, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [27]:
rf = RandomForestRegressor(n_estimators = 50, criterion = 'mae', random_state = 42)
rf.fit(X_train, y_train)

In [28]:
pred_rf = rf.predict(X_test)

err_rf = abs(pred_rf - y_test)

In [29]:
print("Mean Absolute Error:", round(np.mean(err_rf), 2))

In [30]:
df = DecisionTreeRegressor(criterion = 'mae')
model_df = df.fit(X_train, y_train)
pred_df = model_df.predict(X_test)

In [31]:
err_df = abs(pred_df - y_test)
print("Mean Absolute Error:", round(np.mean(err_df), 2))